In [10]:
#Основные библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline 

import dill


#Сплит датасета
from sklearn.model_selection import train_test_split

#Модели
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

#Метрики
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, fbeta_score

In [11]:
def evaluate_preds(true_values, pred_values, pred_proba):
    """Оценка качества модели"""

    print('gini: ', round(2*roc_auc_score(true_values, pred_proba)-1,3))
    print('f1_score: ', round(f1_score(true_values, pred_values),3))
    print('f2_score: ', round(fbeta_score(true_values, pred_values, beta=2),3))
    print('accuracy_score: ', round(accuracy_score(true_values, pred_values),3))
    print('recall_score: ', round(recall_score(true_values, pred_values),3))
    print('precision_score: ', round(precision_score(true_values, pred_values),3))  

In [4]:
def create_probdensity(pred_proba):
    """Распределение вероятностей принадлежности к классу 1 и 0"""
    
    plt.hist(pred_proba[y_valid == 1], bins=7, density=True, label='class 1', alpha=0.5)
    plt.hist(pred_proba[y_valid == 0], bins=7, density=True, label='class 0', alpha=0.5)

    plt.xlabel("Probability")
    plt.ylabel("Density")

    plt.legend()

### I. Загрузка выборок.


Обучающие выборки

In [36]:
train_1 = pd.read_csv('df_train_ADASYN_MinMax.csv')

In [37]:
test_1 = pd.read_csv('df_test_ADASYN_MinMax.csv')

*Процентное соотношение train/test = 72/18*

### II. Работа train_1, test_1

In [38]:
X = train_1.drop(columns='target')
y = train_1['target'].to_numpy()

In [53]:
X_t = test_1.drop(columns=['target', 'client_id', 'dt'])
y_t = test_1['target'].to_numpy()

In [54]:
X_train, X_valid, y_train, y_valid = X,X_t,y, y_t

**Support Vector Classifier**

Построение модели

In [47]:
%%time
n_estimators = 20

svc = BaggingClassifier(SVC(C=10, gamma='auto', random_state=21,  probability=True), n_jobs=25, max_samples=1.0 / n_estimators, n_estimators=n_estimators, bootstrap = False)

svc.fit(X_train, y_train)

CPU times: user 1.69 s, sys: 2.66 s, total: 4.35 s
Wall time: 13min 8s


BaggingClassifier(base_estimator=SVC(C=10, gamma='auto', probability=True,
                                     random_state=21),
                  bootstrap=False, max_samples=0.05, n_estimators=20,
                  n_jobs=25)

In [52]:
X_valid

,client_id,dt,accountsalary_flag,country,profit_income_out_rur_amt_6m,curbal_usd_amt_cm_avg,srvpackage,age,addrref,life_time_days,...,clientsegment_AFFLUENT,clientsegment_MASS,stratsegfactor_Предиктивный доход,stratsegfactor_Реальная зарплата клиента,stratsegfactor_nan,stratsegfactor_Остатки на счетах,stratsegfactor_Зарплата из кредитной анкеты,stratsegfactor_Пакет услуг,stratsegfactor_Инвестиционные продукты,gender_Мужской
0,97f3c73aa27980d8b66369474973b05ff1c93a8cbf5d07...,2022-04-30,0.0,0.0,0.195356,0.000000,0.0,0.297619,0.0,0.005171,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,6846b905c6d77556ab93970fc1ffa651349bd344f9af84...,2021-12-31,0.0,0.0,0.195356,0.000000,0.0,0.369048,0.0,0.035137,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,b31d16a6435112ab5b74005c8579f75743adfdd8a59382...,2022-03-31,1.0,0.0,0.198251,0.000000,0.0,0.309524,0.0,0.347786,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,faef7974c6c6b094578175f09522bf8257a28a167bb81e...,2021-05-31,0.0,0.0,0.195887,0.000000,0.0,0.238095,0.0,0.112039,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,a545e9af11964956d85a5f24710978804e1289c008ec96...,2021-02-28,1.0,0.0,0.195356,0.000000,0.0,0.261905,0.0,0.610846,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46407,e16029cbd8a0326853c2d2126972c68537bf33da408fb1...,2023-01-31,1.0,0.0,0.196882,0.000003,0.0,0.273810,0.0,0.223150,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
46408,ad4030c5671a2699ef69f0101f9450df89abb528ff633c...,2022-03-31,1.0,0.0,0.195372,0.000000,0.0,0.821429,0.0,0.158048,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
46409,492500c3a7a305ee6cde62c179715a1845fb0b7096eb1b...,2022-05-31,1.0,0.0,0.195364,0.000004,0.0,0.083333,0.0,0.089499,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
46410,88141216857e469198701b058d290da73dd05931dc9cfa...,2022-08-31,1.0,0.0,0.198410,0.000043,0.0,0.309524,0.0,0.305887,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
%%time

y_pred_svc = svc.predict(X_valid)
y_pred_proba_svc = svc.predict_proba(X_valid)[:, 1]

/home/ivershin/venv_venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/home/ivershin/venv_venv/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


CPU times: user 1.17 s, sys: 2.25 s, total: 3.43 s
Wall time: 8min 6s


In [56]:
evaluate_preds(y_valid, y_pred_svc, y_pred_proba_svc)

gini:  -0.23
f1_score:  0.108
f2_score:  0.226
accuracy_score:  0.119
recall_score:  0.828
precision_score:  0.058


In [61]:
confusion_matrix(y_valid, y_pred_svc)

array([[ 3042, 40368],
       [  517,  2485]])

In [57]:
#сохраняем лучшую модель
with open('svc.dat', 'wb') as f:
    dill.dump(svc,f)